In [41]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [42]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header= None, engine = 'python', encoding = 'latin-1')

In [43]:
movies

0                                                  1  \
0        1                                   Toy Story (1995)   
1        2                                     Jumanji (1995)   
2        3                            Grumpier Old Men (1995)   
3        4                           Waiting to Exhale (1995)   
4        5                 Father of the Bride Part II (1995)   
5        6                                        Heat (1995)   
6        7                                     Sabrina (1995)   
7        8                                Tom and Huck (1995)   
8        9                                Sudden Death (1995)   
9       10                                   GoldenEye (1995)   
10      11                     American President, The (1995)   
11      12                 Dracula: Dead and Loving It (1995)   
12      13                                       Balto (1995)   
13      14                                       Nixon (1995)   
14      15                            Cutthroat Island (1995)   
15      16                                      Casino (1995)   
16      17                       Sense and Sensibility (1995)   
17      18                                  Four Rooms (1995)   
18      19              Ace Ventura: When Nature Calls (1995)   
19      20                                 Money Train (1995)   
20      21                                  Get Shorty (1995)   
21      22                                     Copycat (1995)   
22      23                                   Assassins (1995)   
23      24                                      Powder (1995)   
24      25                           Leaving Las Vegas (1995)   
25      26                                     Othello (1995)   
26      27                                Now and Then (1995)   
27      28                                  Persuasion (1995)   
28      29                  City of Lost Children, The (1995)   
29      30  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
...    ...                                                ...   
3853  3923                           Return of the Fly (1959)   
3854  3924                                Pajama Party (1964)   
3855  3925                      Stranger Than Paradise (1984)   
3856  3926             Voyage to the Bottom of the Sea (1961)   
3857  3927                            Fantastic Voyage (1966)   
3858  3928       Abbott and Costello Meet Frankenstein (1948)   
3859  3929                              Bank Dick, The (1940)   
3860  3930         Creature From the Black Lagoon, The (1954)   
3861  3931                     Giant Gila Monster, The (1959)   
3862  3932                          Invisible Man, The (1933)   
3863  3933                          Killer Shrews, The (1959)   
3864  3934                                      Kronos (1957)   
3865  3935                                      Kronos (1973)   
3866  3936                   Phantom of the Opera, The (1943)   
3867  3937                                     Runaway (1984)   
3868  3938                 Slumber Party Massacre, The (1982)   
3869  3939              Slumber Party Massacre II, The (1987)   
3870  3940             Slumber Party Massacre III, The (1990)   
3871  3941                     Sorority House Massacre (1986)   
3872  3942                  Sorority House Massacre II (1990)   
3873  3943                                  Bamboozled (2000)   
3874  3944                                     Bootmen (2000)   
3875  3945                          Digimon: The Movie (2000)   
3876  3946                                  Get Carter (2000)   
3877  3947                                  Get Carter (1971)   
3878  3948                            Meet the Parents (2000)   
3879  3949                         Requiem for a Dream (2000)   
3880  3950                                   Tigerland (2000)   
3881  3951                            Two Family House (2000)   
3882  3952                              Cont

In [44]:
users = pd.read_csv('ml-1m/users.dat', sep = '::', header= None, engine = 'python', encoding = 'latin-1')

In [45]:
users

0  1   2   3      4
0        1  F   1  10  48067
1        2  M  56  16  70072
2        3  M  25  15  55117
3        4  M  45   7  02460
4        5  M  25  20  55455
5        6  F  50   9  55117
6        7  M  35   1  06810
7        8  M  25  12  11413
8        9  M  25  17  61614
9       10  F  35   1  95370
10      11  F  25   1  04093
11      12  M  25  12  32793
12      13  M  45   1  93304
13      14  M  35   0  60126
14      15  M  25   7  22903
15      16  F  35   0  20670
16      17  M  50   1  95350
17      18  F  18   3  95825
18      19  M   1  10  48073
19      20  M  25  14  55113
20      21  M  18  16  99353
21      22  M  18  15  53706
22      23  M  35   0  90049
23      24  F  25   7  10023
24      25  M  18   4  01609
25      26  M  25   7  23112
26      27  M  25  11  19130
27      28  F  25   1  14607
28      29  M  35   7  33407
29      30  F  35   7  19143
...    ... ..  ..  ..    ...
6010  6011  M  35  15  80538
6011  6012  M  35  15  02871
6012  6013  F  25  20  32301
6013  6014  M  45   1  80634
6014  6015  F  25   9  80013
6015  6016  M  45   1  37209
6016  6017  F  35   7  21117
6017  6018  M  35   1  48906
6018  6019  M  25   0  10024
6019  6020  M  50  16  10023
6020  6021  M  25  12  08876
6021  6022  M  25  17  57006
6022  6023  M  25   0  43213
6023  6024  M  25  12  53705
6024  6025  F  25   1  32607
6025  6026  M  35   6  11210
6026  6027  M  18   4  20742
6027  6028  M  18   4  94133
6028  6029  F  25   1  23185
6029  6030  M  25  17  32618
6030  6031  F  18   0  45123
6031  6032  M  45   7  55108
6032  6033  M  50  13  78232
6033  6034  M  25  14  94117
6034  6035  F  25   1  78734
6035  6036  F  25  15  32603
6036  6037  F  45   1  76006
6037  6038  F  56   1  14706
6038  6039  F  45   0  01060
6039  6040  M  25   6  11106

[6040 rows x 5 columns]

In [46]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header= None, engine = 'python', encoding = 'latin-1')

In [47]:
ratings

0     1  2          3
0           1  1193  5  978300760
1           1   661  3  978302109
2           1   914  3  978301968
3           1  3408  4  978300275
4           1  2355  5  978824291
5           1  1197  3  978302268
6           1  1287  5  978302039
7           1  2804  5  978300719
8           1   594  4  978302268
9           1   919  4  978301368
10          1   595  5  978824268
11          1   938  4  978301752
12          1  2398  4  978302281
13          1  2918  4  978302124
14          1  1035  5  978301753
15          1  2791  4  978302188
16          1  2687  3  978824268
17          1  2018  4  978301777
18          1  3105  5  978301713
19          1  2797  4  978302039
20          1  2321  3  978302205
21          1   720  3  978300760
22          1  1270  5  978300055
23          1   527  5  978824195
24          1  2340  3  978300103
25          1    48  5  978824351
26          1  1097  4  978301953
27          1  1721  4  978300055
28          1  1545  4  978824139
29          1   745  3  978824268
...       ...   ... ..        ...
1000179  6040  2762  4  956704584
1000180  6040  1036  3  956715455
1000181  6040   508  4  956704972
1000182  6040  1041  4  957717678
1000183  6040  3735  4  960971654
1000184  6040  2791  4  956715569
1000185  6040  2794  1  956716438
1000186  6040   527  5  956704219
1000187  6040  2003  1  956716294
1000188  6040   535  4  964828734
1000189  6040  2010  5  957716795
1000190  6040  2011  4  956716113
1000191  6040  3751  4  964828782
1000192  6040  2019  5  956703977
1000193  6040   541  4  956715288
1000194  6040  1077  5  964828799
1000195  6040  1079  2  956715648
1000196  6040   549  4  956704746
1000197  6040  2020  3  956715288
1000198  6040  2021  3  956716374
1000199  6040  2022  5  956716207
1000200  6040  2028  5  956704519
1000201  6040  1080  4  957717322
1000202  6040  1089  4  956704996
1000203  6040  1090  3  956715518
1000204  6040  1091  1  956716541
1000205  6040  1094  5  956704887
1000206  6040   562  5  956704746
1000207  6040  1096  4  956715648
1000208  6040  1097  4  956715569

[1000209 rows x 4 columns]

In [48]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')

In [49]:
training_set

1   1.1  5  874965758
0        1     2  3  876893171
1        1     3  4  878542960
2        1     4  3  876893119
3        1     5  3  889751712
4        1     7  4  875071561
5        1     8  1  875072484
6        1     9  5  878543541
7        1    11  2  875072262
8        1    13  5  875071805
9        1    15  5  875071608
10       1    16  5  878543541
11       1    18  4  887432020
12       1    19  5  875071515
13       1    21  1  878542772
14       1    22  4  875072404
15       1    25  4  875071805
16       1    26  3  875072442
17       1    28  4  875072173
18       1    29  1  878542869
19       1    30  3  878542515
20       1    32  5  888732909
21       1    34  2  878542869
22       1    35  1  878542420
23       1    37  2  878543030
24       1    38  3  878543075
25       1    40  3  876893230
26       1    41  2  876892818
27       1    42  5  876892425
28       1    43  4  878542869
29       1    45  5  875241687
...    ...   ... ..        ...
79969  943   720  1  888640048
79970  943   721  5  888639660
79971  943   722  3  888640208
79972  943   724  1  888639478
79973  943   732  4  888639789
79974  943   739  4  888639929
79975  943   756  2  875502146
79976  943   763  4  875501813
79977  943   765  3  888640227
79978  943   785  2  888640088
79979  943   794  3  888640143
79980  943   796  3  888640311
79981  943   808  4  888639868
79982  943   816  4  888640186
79983  943   824  4  875502483
79984  943   825  3  875502283
79985  943   831  2  875502283
79986  943   840  4  888693104
79987  943   928  5  875502074
79988  943   941  1  888639725
79989  943   943  5  888639614
79990  943  1011  2  875502560
79991  943  1028  2  875502096
79992  943  1044  3  888639903
79993  943  1047  2  875502146
79994  943  1067  2  875501756
79995  943  1074  4  888640250
79996  943  1188  3  888640250
79997  943  1228  3  888640275
79998  943  1330  3  888692465

[79999 rows x 4 columns]

In [50]:
training_set = np.array(training_set, dtype = 'int')

In [51]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

In [52]:
test_set = np.array(test_set, dtype = 'int')

In [53]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [54]:
nb_users, nb_movies

(943, 1682)

In [55]:
def convert(data):
    new_data = []
    for id_users in range(1,nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [56]:
training_set = convert(training_set)
test_set = convert(test_set)

In [57]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [58]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [59]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [60]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [61]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [66]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [67]:
nv = len(training_set[0])
nh = 100
batch_size = 100

rbm = RBM(nv, nh)

In [70]:
nb_epoch = 20
for epoch in range(1,nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_users in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_users:id_users+batch_size]
        v0 = training_set[id_users:id_users+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+ '    Loss: '+str(train_loss/s))

epoch: 1    Loss: tensor(0.2479)
epoch: 2    Loss: tensor(0.2499)
epoch: 3    Loss: tensor(0.2470)
epoch: 4    Loss: tensor(0.2468)
epoch: 5    Loss: tensor(0.2486)
epoch: 6    Loss: tensor(0.2485)
epoch: 7    Loss: tensor(0.2440)
epoch: 8    Loss: tensor(0.2473)
epoch: 9    Loss: tensor(0.2464)
epoch: 10    Loss: tensor(0.2464)
epoch: 11    Loss: tensor(0.2485)
epoch: 12    Loss: tensor(0.2417)
epoch: 13    Loss: tensor(0.2487)
epoch: 14    Loss: tensor(0.2462)
epoch: 15    Loss: tensor(0.2470)
epoch: 16    Loss: tensor(0.2477)
epoch: 17    Loss: tensor(0.2450)
epoch: 18    Loss: tensor(0.2465)
epoch: 19    Loss: tensor(0.2526)
epoch: 20    Loss: tensor(0.2473)


In [72]:
test_loss = 0
s = 0.
for id_users in range(0, nb_users):
    v = training_set[id_users:id_users+1]
    vt = test_set[id_users:id_users+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('Test Loss: '+str(test_loss/s))

Test Loss: tensor(0.2380)
